In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

sys.path.insert(0, '..')
from data_wrangling import create_new_folder, get_image_info
from preprocessing import delete_corrupt_image

In [41]:
train = r'../train_images'
val = r'../val_images'
test = r'../test_images'

In [42]:
train_df = pd.read_csv(r'..\data\train.csv')
train_df.drop(labels='Unnamed: 0', axis=1,inplace=True)

In [43]:
delete_corrupt_image(train)
delete_corrupt_image(val)
delete_corrupt_image(test)

Deleted 0 images
Deleted 0 images
Deleted 0 images


In [44]:
train_df.label.value_counts()

Impressionism             616
Cubism                    198
Expressionism              87
Pop Art                    82
Byzantine Art              44
Abstract Expressionism     11
Name: label, dtype: int64

In [45]:
#create generator for image augmentation
data_gen = ImageDataGenerator(rotation_range=40, 
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              brightness_range=(0.25, 1.0),
                              channel_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2, 
                              horizontal_flip=True, 
                              fill_mode='reflect')

In [46]:
#augment data and add samples to smaller classes in training images only
#find max number of images for a genre 
maximum = max(train_df.label.value_counts())

for i in train_df.label.unique():
    num = int(train_df.label[train_df.label == i].value_counts())
    if num < maximum:
        sample = list(train_df.filepath.loc[train_df.label == i].sample(frac=.5, replace=False))
        for s in sample:
            img = load_img(s)
            x = img_to_array(img)
            x = x.reshape((1,) +x.shape)
            count=0
            
            for batch in data_gen.flow(x, batch_size=1, save_to_dir=os.path.join(train,i),
                                      save_prefix=f'{i}_aug', save_format='jpeg'):
                count+=1
                if count>0:
                    break

In [47]:
#get image information from train.csv
train_aug_df = get_image_info(train)
#save training samples to train.csv
train_aug_df.to_csv(r'..\data\train_augmented.csv')

In [48]:
train_aug_df.label.value_counts().sum()

Impressionism             616
Cubism                    297
Expressionism             131
Pop Art                   123
Byzantine Art              66
Abstract Expressionism     17
Name: label, dtype: int64

In [49]:
train_aug_df.label.value_counts().sum()

1250